In [17]:
import os
import pandas as pd

In [29]:
csv_dir = "csv"
folders = [folder for folder in os.listdir(csv_dir) if "HuBMAP" in folder or "GTEx" in folder or "CxG" in folder]

number of datasets

In [30]:
count = 0
for folder in folders:
    if folder != ".DS_Store":
        count += len(os.listdir(os.path.join(csv_dir,folder)))
print(count)

3366


number of organs

In [31]:
organs = []
for folder in folders:
    if folder != ".DS_Store":
        organs.append(" ".join(folder.split("_")[:-2]))
print(len(set(organs)))

52


number of cell types

In [32]:
cell_types = []
for folder in folders:
    if folder != ".DS_Store":
        for csv_file in os.listdir(os.path.join(csv_dir,folder)):
            data = pd.read_csv(os.path.join(csv_dir,folder,csv_file))
            cell_types.extend(data['cell_type'].unique())
print(len(set(cell_types)))

643
